In [1]:
import findspark
findspark.init()
from pyspark import SparkContext

sc = SparkContext("local[10]","bruted_force")

input = sc.textFile('./pagerank_data.txt')

In [2]:
data = input.map(lambda x: [int(temp) for temp in x.split(' ')]).map(lambda x: (x[0],x[1:]))

In [54]:
block_1 = data.map(lambda x: (x[0],x[1][:len(x[1])//3]))
block_2 = data.map(lambda x: (x[0],x[1][len(x[1])//3:2*(len(x[1])//3)]))
block_3 = data.map(lambda x: (x[0],x[1][2*(len(x[1])//3):]))

In [53]:
def generate_block(N):
  temp = data
  for i in range(N):
    if(i==0):
      temp = data.map(lambda x: (x[0],x[1][:(len(x[1])//N)]))
    elif (i==N-1):
      temp += data.map(lambda x: (x[0],x[1][i*(len(x[1])//N):]))
    else:
      temp += data.map(lambda x: (x[0],x[1][i*(len(x[1])//N):(i+1)*(len(x[1])//N)]))
  return temp

In [55]:
temp = generate_block(3)
block_arr = block_1+block_2+block_3

In [56]:
print(len(temp.first()[1]),len(block_arr.first()[1]))

333 333


In [98]:
def cul(N,maxIter):
  convergenceTol = 0.0001
  d= 0.85
  block_arr = generate_block(N);
  pr = sc.broadcast([1./float(30000) for _ in range(30000)])
  for _ in range(maxIter):
    iter = block_arr.flatMap(lambda t : [(target,(pr.value[t[0]]) / ((len(t[1])*N))) for target in t[1]])\
            .reduceByKey(lambda x,y : x+y).mapValues(lambda v : d*v).cache()

    temp = data.map(lambda x: (x[0],1/(30000)*(1-d)))
    result = temp.union(iter).reduceByKey(lambda x,y:x+y).sortByKey()
    err = result.map(lambda t: abs(pr.value[t[0]] - t[1])).sum()/N
    print(err)
    if err < convergenceTol:
      pr = result.map(lambda t:t[1]).collect()
      return pr
    pr = sc.broadcast(result.map(lambda t:t[1]).collect())
  return pr


In [ ]:
result = cul(10,20)

In [88]:
result.value[:30]

[5.000000000000001e-06,
 0.000855386546912898,
 0.000855386546912898,
 0.000855386546912898,
 0.000855386546912898,
 0.000855386546912898,
 0.000855386546912898,
 0.000855386546912898,
 0.000855386546912898,
 0.000855386546912898,
 0.000855386546912898,
 0.000855386546912898,
 0.000855386546912898,
 0.000855386546912898,
 0.000855386546912898,
 0.0008553865469128976,
 0.0008553865469128979,
 0.0008553865469128979,
 0.0008553865469128979,
 0.000855386546912898,
 0.000855386546912898,
 0.0008553865469128979,
 0.0008553865469128979,
 0.0008553865469128979,
 0.000855386546912898,
 0.000855386546912898,
 0.000855386546912898,
 0.000855386546912898,
 0.0008553865469128979,
 0.000855386546912898]

In [83]:
temp = generate_block(3)
pr = sc.broadcast([1./float(30000) for _ in range(30000)])
N=3
d = 0.85

In [85]:
iter.take(30)

[(1, 0.000855386546912898),
 (2, 0.000855386546912898),
 (3, 0.000855386546912898),
 (4, 0.000855386546912898),
 (5, 0.000855386546912898),
 (6, 0.000855386546912898),
 (7, 0.000855386546912898),
 (8, 0.000855386546912898),
 (9, 0.000855386546912898),
 (10, 0.000855386546912898),
 (11, 0.000855386546912898),
 (12, 0.000855386546912898),
 (13, 0.000855386546912898),
 (14, 0.000855386546912898),
 (15, 0.0008553865469128976),
 (16, 0.0008553865469128979),
 (17, 0.0008553865469128979),
 (18, 0.0008553865469128979),
 (19, 0.000855386546912898),
 (20, 0.000855386546912898),
 (21, 0.0008553865469128979),
 (22, 0.0008553865469128979),
 (23, 0.0008553865469128979),
 (24, 0.000855386546912898),
 (25, 0.000855386546912898),
 (26, 0.000855386546912898),
 (27, 0.000855386546912898),
 (28, 0.0008553865469128979),
 (29, 0.000855386546912898),
 (30, 0.0008553865469128976)]